In [1]:
%%capture
!pip install bertopic accelerate bitsandbytes xformers adjustText ja-sentence-tokenizer mecab-python3 unidic-lite
!{sys.executable} -m pip install "scipy==1.9.0" "scikit-image==0.23.2"

import os
import sys
import pandas as pd
from multiling_helpers import JapaneseProcessor, TopicModeler  # Import your classes

# Setup paths
current_dir = os.getcwd()
lyra_dir = os.path.abspath(os.path.join(current_dir, '..'))
print("Lyra directory:", lyra_dir)

os.chdir(lyra_dir)
sys.path.insert(0, lyra_dir)

# Load data
condition = "innerspeech_japanese" #dataset
language = "japanese"
reports_path = os.path.join("DATA", f"{condition}_reduced.csv")
df = pd.read_csv(reports_path)


In [2]:
# Initialize Japanese processor and topic modeler
japanese_processor = JapaneseProcessor()
#print the infos about the processor for japanese
japanese_processor.print_info()

japanese_processor.sentence_transformer_model = "paraphrase-multilingual-mpnet-base-v2"
japanese_modeler = TopicModeler(japanese_processor) #iniy modeler with modified processor

Language: japanese
Transformer model: bge-multilingual-gemma2
Number of stopwords: 38
Sentence endings: {'。', '！', '\n', '？', '!', '．', '.', '?'}


/home/romy/lyra/.colabvenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
#data processing
# column with text of interst stotred in "reflection_answer"
text_column = 'reflection_answer'
data = df[text_column].dropna().tolist()

# Preprocess and split into sentences
print(f"Processing {len(data)} {language} reports...")
processed_data = japanese_modeler.process_data(data, split_sentences=True)
print(f"Processed into {len(processed_data)} sentences.")
# Prepare vectorizer
vectorizer = japanese_modeler.prepare_vectorizer()

# Get embeddings
embeddings = japanese_modeler.embedding_model.encode(processed_data, show_progress_bar=True)

Processing 731 japanese reports...
Processed into 1939 sentences.


Batches:   0%|          | 0/61 [00:00<?, ?it/s]

In [4]:
data[:10]

['頭の中の独り言をこのような調査で改めて自覚することができ、また色々なパターンがあることを知り面白かった。',
 '他人の声が脳内でしている人がいるという話にすごく興味があるのですが、心理物理実験で音声のパラメータチューニングかA/Bテストかを繰り返すことでその人の脳内の声を具現化することってできますかね？（具現化しようとした途端に霞をつかむようにとらえどころのないものになってしまうなどの可能性もあるかもなあと思いつつ）',
 '頭の中では日本語で考えているという自覚はある（英語は勉強以外にほぼ使ったことはない）が、文字か音声かと言われると、どちらも混ざっている感じがする。\n映像や味などは思い出される事がある。',
 '自動思考というものなのか、直近で起きた失敗などを批判する考えが勝手に浮かんできたりすることがあるが、自分の声とか他人の声という認識はしていなかった。',
 '食べたいもの、欲しいものなどは、自分がそれを食べている、或いは使っているところを想像して決めているような感じでした。逆に、どこに置いたかな、というのは「自分が置いたときのイメージが湧かないからこそそういう状況になっている」ときは言葉で、「場面を辿ればわかる」ときは映像で思い浮かぶ感じでした。\n発達系の話になりますが視覚優位か聴覚優位かなどによっても変わりそうで面白いなと思いました。',
 '画像や映像が頭の中に流れ、その後に自分の声で考える事が多い。',
 'フローチャートや箇条書きで考えるのは便利そうなので意識的にやってみたい。',
 'キャスト全員が自分の声で話しているボイスドラマを聴いている風に内言します。',
 '思考中にふと反対意見・対立意見が他人の声で聞こえる',
 '言葉にできないことが多い。説明や言葉が足りず、内容を人に理解させることが難しい。']

### BERTopic

In [10]:
#set up UMAP and HDBSCAN for topic modelling
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic

umap_model = UMAP(
    n_neighbors=15,
    n_components=3,
    min_dist=0.01,
    metric='cosine',
    random_state=42
)

hdbscan_model = HDBSCAN(
    min_cluster_size=20,
    min_samples=None,
    gen_min_span_tree=True,
    prediction_data=True
)

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    language="japanese",
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(processed_data, embeddings)


2024-12-03 12:12:58,444 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-03 12:13:02,167 - BERTopic - Dimensionality - Completed ✓
2024-12-03 12:13:02,167 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-03 12:13:02,218 - BERTopic - Cluster - Completed ✓
2024-12-03 12:13:02,220 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-03 12:13:02,231 - BERTopic - Representation - Completed ✓


In [11]:
# Get topic info
topic_info = topic_model.get_topic_info()
print("Number of topics found:", len(topic_info)-1)  # -1 to exclude outlier topic
print("\nTop topics by size:")
print(topic_info.head())

# Get top words for each topic
for topic in topic_info.Topic:
    if topic != -1:  # Skip outlier topic
        print(f"\nTopic {topic}:")
        print(topic_model.get_topic(topic))

Number of topics found: 3

Top topics by size:
   Topic  Count                  Name  \
0     -1     34      -1_頑張ってください_言語__   
1      0   1707         0_また_映像_ただ_文章   
2      1     83  1_面白かったです_頑張ってください__   
3      2     22        2_内言は_文章_ただ_映像   

                             Representation  \
0            [頑張ってください, 言語, , , , , , , , ]   
1  [また, 映像, ただ, 文章, 言語, 内言は, 面白かったです, , , ]   
2       [面白かったです, 頑張ってください, , , , , , , , ]   
3           [内言は, 文章, ただ, 映像, また, , , , , ]   

                                 Representative_Docs  
0  [応援しています。, 頑張ってください。, ADHD気味で、言語(大脳?)で考えるというより...  
1  [まるで映画のような)\nまた、人と会話をしている際に内言が頭の中でする時間は、人との会話を...  
2                      [面白かったです, 面白かったです。, 面白かったです。]  
3  [ただ、夢を見ている時などは他人の声もかなり正確に再現することが出来たり、なにより自分とは切...  

Topic 0:
[('また', 0.23368575668089628), ('映像', 0.17380756940472192), ('ただ', 0.16156675953135058), ('文章', 0.0648636716351771), ('言語', 0.03837641821656743), ('内言は', 0.03837641821656743), ('面白かったです', 0.025434271724917485), ('', 1e-05), (''

In [12]:
topic_model.visualize_barchart()


In [13]:
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.01, random_state=42).fit_transform(embeddings)
topic_model.visualize_documents(
    docs=processed_data,
    reduced_embeddings=reduced_embeddings  # Pass reduced embeddings
)